In [29]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

train_data = pd.read_csv('data/train.csv', index_col='Id')
X_test = test_data = pd.read_csv('data/test.csv', index_col='Id')

In [30]:
X_train, y = train_data.iloc[:, :-1], train_data.iloc[:, -1]
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal


In [31]:
X = pd.concat([X_train, X_test])
X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


In [44]:
X.isnull().sum()

MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 79, dtype: int64

In [43]:
X.shape

(2919, 79)

In [32]:
import torch

columns = torch.load('data/feature_col.pkl')
num_col, class_col, ordering_col = columns

In [33]:
X_num = X[num_col]
X_class = X[class_col]
X_num

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,BedroomAbvGr,KitchenAbvGr
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,...,61,0,0,0,0,0,2,2008,3,1
2,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,...,0,0,0,0,0,0,5,2007,3,1
3,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,...,42,0,0,0,0,0,9,2008,3,1
4,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,...,35,272,0,0,0,0,2,2006,3,1
5,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,...,84,0,0,0,0,0,12,2008,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,...,0,0,0,0,0,0,6,2006,3,1
2916,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,...,24,0,0,0,0,0,4,2006,3,1
2917,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,...,0,0,0,0,0,0,9,2006,4,1


In [34]:
# 数量的处理
X_class = pd.get_dummies(X_class, dummy_na=True)
X_class

,MSSubClass,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MSZoning_nan,Street_Grvl,Street_Pave,Street_nan,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
Id,,,,,,,,,,,,,,,,,,,,,
1,60,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,20,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
3,60,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,70,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
5,60,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2916,160,0,0,0,0,1,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
2917,20,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0


In [35]:
from sklearn.impute import SimpleImputer

num_imp = SimpleImputer(strategy='mean')
X_num = pd.DataFrame(data=num_imp.fit_transform(X_num), columns=X_num.columns, index=X_num.index)
X_num

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,BedroomAbvGr,KitchenAbvGr
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,61.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,3.0,1.0
2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,3.0,1.0
3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,3.0,1.0
4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,3.0,1.0
5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,546.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2006.0,3.0,1.0
2916,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,294.0,...,24.0,0.0,0.0,0.0,0.0,0.0,4.0,2006.0,3.0,1.0
2917,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,4.0,1.0


In [52]:
# 定序变量的处理
X_order = X[ordering_col.keys()]



In [54]:
has_NA_col = []
no_has_NA = []
for key, rank in ordering_col.items():
    if 'NA' in rank:
        has_NA_col.append(key)
    else:
        no_has_NA.append(key)

has_NA_col, no_has_NA

(['BsmtQual',
  'BsmtCond',
  'BsmtExposure',
  'BsmtFinType1',
  'BsmtFinType2',
  'FireplaceQu',
  'GarageFinish',
  'GarageQual',
  'GarageCond',
  'PoolQC',
  'Fence'],
 ['LotShape',
  'LandContour',
  'Utilities',
  'LandSlope',
  'ExterQual',
  'ExterCond',
  'HeatingQC',
  'KitchenQual',
  'Functional',
  'PavedDrive'])

In [55]:
X_order_NA = X_order[has_NA_col]
X_order_noNA = X_order[no_has_NA]

In [58]:
X_order_NA.fillna('NA', inplace=True)

d:\project\python\.venv\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [59]:
X_order_NA.isnull().sum()

BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
dtype: int64

In [60]:
from sklearn.impute import SimpleImputer

fre_imp = SimpleImputer(strategy='most_frequent')
X_order_noNA = pd.DataFrame(data=fre_imp.fit_transform(X_order_noNA), columns=X_order_noNA.columns,
                            index=X_order_noNA.index)
X_order_noNA.isnull().sum()

LotShape       0
LandContour    0
Utilities      0
LandSlope      0
ExterQual      0
ExterCond      0
HeatingQC      0
KitchenQual    0
Functional     0
PavedDrive     0
dtype: int64

In [61]:
X_order = pd.concat([X_order_noNA, X_order_NA], axis=1)
X_order

,LotShape,LandContour,Utilities,LandSlope,ExterQual,ExterCond,HeatingQC,KitchenQual,Functional,PavedDrive,...,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence
Id,,,,,,,,,,,,,,,,,,,,,
1,Reg,Lvl,AllPub,Gtl,Gd,TA,Ex,Gd,Typ,Y,...,TA,No,GLQ,Unf,NA,RFn,TA,TA,NA,NA
2,Reg,Lvl,AllPub,Gtl,TA,TA,Ex,TA,Typ,Y,...,TA,Gd,ALQ,Unf,TA,RFn,TA,TA,NA,NA
3,IR1,Lvl,AllPub,Gtl,Gd,TA,Ex,Gd,Typ,Y,...,TA,Mn,GLQ,Unf,TA,RFn,TA,TA,NA,NA
4,IR1,Lvl,AllPub,Gtl,TA,TA,Gd,Gd,Typ,Y,...,Gd,No,ALQ,Unf,Gd,Unf,TA,TA,NA,NA
5,IR1,Lvl,AllPub,Gtl,Gd,TA,Ex,Gd,Typ,Y,...,TA,Av,GLQ,Unf,TA,RFn,TA,TA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,Reg,Lvl,AllPub,Gtl,TA,TA,Gd,TA,Typ,Y,...,TA,No,Unf,Unf,NA,NA,NA,NA,NA,NA
2916,Reg,Lvl,AllPub,Gtl,TA,TA,TA,TA,Typ,Y,...,TA,No,Rec,Unf,NA,Unf,TA,TA,NA,NA
2917,Reg,Lvl,AllPub,Gtl,TA,TA,Ex,TA,Typ,Y,...,TA,No,ALQ,Unf,TA,Unf,TA,TA,NA,NA


In [62]:
for key, rank in ordering_col.items():
    X_order[key] = X_order[key].map(lambda x: rank.index(x))

X_order

,LotShape,LandContour,Utilities,LandSlope,ExterQual,ExterCond,HeatingQC,KitchenQual,Functional,PavedDrive,...,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence
Id,,,,,,,,,,,,,,,,,,,,,
1,3,3,3,2,3,2,4,3,7,2,...,3,1,6,1,0,2,3,3,0,0
2,3,3,3,2,2,2,4,2,7,2,...,3,4,5,1,3,2,3,3,0,0
3,2,3,3,2,3,2,4,3,7,2,...,3,2,6,1,3,2,3,3,0,0
4,2,3,3,2,2,2,3,3,7,2,...,4,1,5,1,4,1,3,3,0,0
5,2,3,3,2,3,2,4,3,7,2,...,3,3,6,1,3,2,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,3,3,3,2,2,2,3,2,7,2,...,3,1,1,1,0,0,0,0,0,0
2916,3,3,3,2,2,2,2,2,7,2,...,3,1,3,1,0,1,3,3,0,0
2917,3,3,3,2,2,2,4,2,7,2,...,3,1,5,1,3,1,3,3,0,0


In [63]:
X = pd.concat([X_num, X_class, X_order], axis=1)
X

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,3,1,6,1,0,2,3,3,0,0
2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,3,4,5,1,3,2,3,3,0,0
3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,3,2,6,1,3,2,3,3,0,0
4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,4,1,5,1,4,1,3,3,0,0
5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,3,3,6,1,3,2,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,546.0,...,3,1,1,1,0,0,0,0,0,0
2916,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,294.0,...,3,1,3,1,0,1,3,3,0,0
2917,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,0.0,...,3,1,5,1,3,1,3,3,0,0


In [71]:
X_y = pd.concat([X, y], axis=1)
plt.figure(figsize=(10, 10))
X_y.corr()[['SalePrice']]

,SalePrice
LotFrontage,0.334820
LotArea,0.263843
OverallQual,0.790982
OverallCond,-0.077856
YearBuilt,0.522897
...,...
GarageQual,0.273839
GarageCond,0.263191
PoolQC,0.115484
Fence,-0.146942


<Figure size 720x720 with 0 Axes>

In [72]:
torch.save([X, y], 'data/house_price.pkl')